In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
df_ratings = pd.read_csv('./inputs/rating_final.csv')

In [3]:
df_geoplaces = pd.read_csv('./inputs/geoplaces2.csv')

In [4]:
df_ratings['overall_rating'] = (df_ratings['rating'] + df_ratings['food_rating'] + df_ratings['service_rating'])

In [5]:
df = pd.merge(df_ratings, df_geoplaces, on='placeID')

In [6]:
user_ids = df['userID'].tolist()
user_ids = list(map(lambda id : int(id[1:])-1001, user_ids))

In [7]:
df['userID'] = pd.DataFrame(user_ids) 

In [8]:
place_ids = df['placeID'].tolist()
place_ids.sort()
place_ids = list(map(lambda id : (id-132560), place_ids))

new_place_ids = []
i = -1
seen = []

for place_id in place_ids:
    if place_id not in seen:
        seen.append(place_id)
        i = i+1
    new_place_ids.append(i)

In [9]:
df['placeID'] = pd.DataFrame(new_place_ids) 

In [10]:
df = df[['userID', 'placeID', 'overall_rating', 'name']]
df

,userID,placeID,overall_rating,name
0,76,0,6,Tortas Locas Hipocampo
1,107,0,4,Tortas Locas Hipocampo
2,80,0,4,Tortas Locas Hipocampo
3,55,0,6,Tortas Locas Hipocampo
4,133,1,5,Tortas Locas Hipocampo
...,...,...,...,...
1156,60,128,6,tacos los volcanes
1157,24,129,1,tacos los volcanes
1158,96,129,4,tacos los volcanes
1159,95,129,5,tacos los volcanes


In [11]:
n_users = df.userID.nunique()
n_items = df.placeID.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Restaurants: '+str(n_items))

Num. of Users: 138
Num of Restaurants: 130


In [19]:
train_data, test_data = train_test_split(df, test_size=0.25)

In [20]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1], line[2]] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1], line[2]] = line[3]

In [14]:
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [15]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [16]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [17]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [18]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 4.297438844018321
Item-based CF RMSE: 4.330152077238667
